# Twomes interactive inverse grey-box analysis pipeline

This Jupyter Labs notebook can be used to interactively test the Twomes inverse grey-box analysis pipeline, accessing data from a Twomes database (see also [more information how to setup a Twomes server](https://github.com/energietransitie/twomes-backoffice-configuration#jupyterlab)).
Don't forget to install the requirements listed in [requirements.txt](../requirements.txt) first!



## Setting the stage

First several imports and variables need to be defined


### Imports and generic settings

In [3]:
from datetime import datetime, timedelta
import pytz
import math
import pylab as plt

import pandas as pd
import numpy as np

# usually, two decimals suffice for displaying DataFrames (NB internally, precision may be higher)
pd.options.display.precision = 2

import sys
sys.path.append('../data/')
sys.path.append('../view/')
sys.path.append('../analysis/')

%load_ext autoreload

%matplotlib widget
from plotter import Plot
from filewriter import ExcelWriter as ex

from extractor import WeatherExtractor, Extractor, Period

from inversegreyboxmodel import Learner


import logging
logger = logging.getLogger('Twomes data extraction')
logger.setLevel(logging.NOTSET)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Analysis settings

- which `learn_duration` should be used for the analysis
- and various other global parameters

In [ ]:
n_std_outliers = 3.0 # default for the multiplier of the the standard deviation; further out than this times the std, outliers are removed during preprocessing
up_intv = '5min' # the default upsampling interval that is used before interpolation is done
gap_n_intv = 11 # the default maximum number of consecutive NaNs to fill(one for each upsampling interval), i.e. valid measurement values (11+1)* 5 min = 1 hour apart apart will be bridget by interpolation, but not more
sampling_interval = '15min' # the default interval on which interpolation will be done during preprocessing
learn_duration_d = 7
required_columns_for_sanity = ['temp_out__degC', 'wind__m_s_1','ghi__W_m_2', 'temp_in__degC', 'g_use__W', 'e_use__W', 'e_ret__W']
sanity_threshold_timedelta = timedelta(hours=24)

### Defining which homes, which period 

- which `homes` should be analysed
- what the location and timezone is of those homes (currently, we only support one location and timezone for a batch of homes) 

- from which `start_day` to which `end_day'  the analysis should run

In [ ]:
#location: center of Assendorp neighbourhood in Zwolle
lat, lon = 52.50655, 6.09961


#timezone: 
timezone_database = 'UTC'
timezone_homes = 'Europe/Amsterdam'


# Below, the maximum period for data collection
first_day = pytz.timezone(timezone_homes).localize(datetime(2021, 10, 25))
last_day = pytz.timezone(timezone_homes).localize(datetime(2022, 5, 8))

# Alternatively, you may want to test things only on a three week periode. This is a period with suitable weather and lots of homes with measurements.
# first_day = pytz.timezone(timezone_homes).localize(datetime(2022, 1, 3))
# last_day = pytz.timezone(timezone_homes).localize(datetime(2022, 1, 31))

# The full set of homes
homes = [803422, 805164, 809743, 811308, 815925, 817341, 822479, 829947, 830088, 831062, 839440, 845966, 845997, 846697, 857477, 864296, 873985, 879481, 881611, 886307, 895671, 897349, 899510]

# # A subset of homes
# homes = [803422, 805164, 809743]

# single home for virtual homes
# homes = [886307]

# single home for gap assessment
# homes = [803422]

## Loading and geospatial interpolation of Dutch weather data

Using an external library installaed via [requirements.txt](../requirements.txt), load and geospatially interpolate Dutch weather data


In [ ]:
%%time 
%autoreload 2

# get geospatially interpolated weather from KNMI
# for Twomes, the Weather for all all homes studies can be approached by a single location
# get the dataframe only once for all homes to save time
tz_knmi='Europe/Amsterdam'

df_weather = WeatherExtractor.get_interpolated_weather_nl(first_day, last_day, lat, lon, tz_knmi, timezone_homes, sampling_interval)

### Check descriptive statisctics about the weather data

In [ ]:
df_weather.info()

In [ ]:
df_weather.describe(include='all')

## Getting time-interpolated home data from the Twomes database and combine with weather data

In [ ]:
%%time 

logger.setLevel(logging.INFO)

df_data_homes = Extractor.get_preprocessed_homes_data(homes, first_day, last_day, timezone_database, timezone_homes,
                                                      up_intv, gap_n_intv, sampling_interval, 
                                                      df_weather)
logger.setLevel(logging.NOTSET)

### Optional block to get interpolated data from virtual homes in CSV files and combine with weather data already obtained


In [ ]:
# %%time 
# %autoreload 2
# logger.setLevel(logging.INFO)

# homes = [
#     60200, 
#     120100, 
#     150080, 
#     150100, 
#     200060, 
#     300040, 
#     400030, 
#     600020 
# ]

# # For virtual homes, only the following period is valid:
# first_day = pytz.timezone(timezone_homes).localize(datetime(2022, 1, 3))
# last_day = pytz.timezone(timezone_homes).localize(datetime(2022, 1, 24))

# df_data_homes = pd.DataFrame()
# for home_id in homes:
#     df_data_homes = pd.concat([df_data_homes, Extractor.get_virtual_home_data_csv(str('../data/virtualhome_P{0}.csv'.format(home_id)), timezone_homes)], axis=0)

# logger.setLevel(logging.NOTSET)

In [ ]:
df_data_homes

## Learn parameters using inverse grey-box analysis

Most of the heavy lifting is done by the `learn_home_parameters()` function, which again uses the [GEKKO Python](https://machinelearning.byu.edu/) dynamic optimization toolkit.

In [ ]:
%%time 
%autoreload 2

# Use one of the lines below to set the moving horizon duration used for analysis 
# learn_duration_d_analysis = 14
learn_duration_d_analysis = learn_duration_d


# learn the model parameters and write rerults an intermediate results to excel files
df_results_model_parameters, df_results_tempsim = Learner.learn_home_parameters(df_data_homes, 
                                                         n_std_outliers, up_intv, gap_n_intv, sampling_interval, 
                                                         learn_duration_d_analysis, 
                                                         req_col = required_columns_for_sanity, sanity_threshold_timedelta = sanity_threshold_timedelta,
                                                         hint_A_m2=None, ev_type=2)

## Show the results

### Show learned model parameters

#### Show table of all learned model parameters of all homes

In [ ]:
df_results_model_parameters

#### Visualize results of all learned model parameters of all homes in one plot

In [ ]:
%autoreload 2

Plot.learned_parameters_boxplot('Learned model parameters for homes', df_results_model_parameters)

#### Visualize results of all learned model parameters by week for each home multiple plots

In [ ]:
%autoreload 2

Plot.learned_parameters_plot(df_results_model_parameters)

### Show best fitting simulated temperatures and power flows

In [ ]:
units_to_mathtext = property_types = {
    'degC' : r'$°C$',
    'W' : r'$W$',
    'W_m_2' : r'$W\cdotm^{-1}$'
}

#### Show table of best fitting simulated temperatures


In [ ]:
df_results_tempsim

#### Show a plot with the best fitting simulated temperatures and power flows

In [ ]:
%autoreload 2

Plot.dataframe_properties_plot(df_results_tempsim,units_to_mathtext)
